In [2]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls = ['https://www.bentleymotors.com/en/your-bentley/accessories.html','https://www.bentleymotors.com/en/about-bentley/history-and-heritage.html','https://www.bentleymotors.com/en/models.html#card-item-d6be51a31b','https://www.bentleymotors.com/en/models/bentley-hybrids.html']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()  

In [2]:
data

[Document(metadata={'source': 'https://www.bentleymotors.com/en/your-bentley/accessories.html'}, page_content="Bentley Accessories\n\nEvery Bentley owner is different. That’s why we offer an extensive range of Bentley car accessories, designed to enhance the way your car looks, sounds and feels, elevating your driving experience. In short, there is no better way to complete your vision and make your Bentley unique.\n\nCONTACT official BENTLEY dealer create your own\n\nCrafted to complement your car\n\nWhatever accessories you choose, however, you can expect a seamless continuation of the Bentley design language. For example, all carbon fibre styling enhancements are mirror-matched using the same process as the carbon fibre veneers inside your car, for absolute symmetry, right down to the weave. And because every accessory is made by Bentley, each one is rigorously tested and inspected to the same exacting standards as your car. They are also supplied with a warranty of at least two yea

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  24


In [4]:
docs[6]

Document(metadata={'source': 'https://www.bentleymotors.com/en/about-bentley/history-and-heritage.html'}, page_content='The Bentley Story\n\nOur story\n\nFrom W.O. Bentley, who founded Bentley Motors in 1919, to the current team of over 4,000 dedicated employees, the company’s extraordinary cars have always been designed and built by exceptional people using only the finest of materials. They have always been driven by exceptional people, too. From the passionate Bentley Boys and Girls who raced the cars in the 1920s, encouraging W.O. Bentley to achieve ever greater feats of engineering, to the visionary Bentley owners of today, Bentley drivers help to shape the world around them.\n\nAs a brand we are also constantly looking forwards, and in this section you will find our technological innovations and future vision for Bentley. And now we are going further, towards a more sustainable future. In 2020 Bentley Motors launched the ‘Beyond100’ Manifesto, which outlines our intention to beco

In [3]:
import os
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
GOOGLE_API_KEY='AJ...' # use own Key
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

# Embed a query
vector = embeddings.embed_query("hello, world!")
print(vector[:5])

C:\Users\haris\miniconda3\envs\env_langchain1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.05168594419956207, -0.030764883384108543, -0.03062233328819275, -0.02802734449505806, 0.01813092641532421]


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retrieved_docs = retriever.invoke("What service you provide")

In [9]:
len(retrieved_docs)

5

In [10]:
print(retrieved_docs[0].page_content)

The Bentley Story

Our story

From W.O. Bentley, who founded Bentley Motors in 1919, to the current team of over 4,000 dedicated employees, the company’s extraordinary cars have always been designed and built by exceptional people using only the finest of materials. They have always been driven by exceptional people, too. From the passionate Bentley Boys and Girls who raced the cars in the 1920s, encouraging W.O. Bentley to achieve ever greater feats of engineering, to the visionary Bentley owners of today, Bentley drivers help to shape the world around them.

As a brand we are also constantly looking forwards, and in this section you will find our technological innovations and future vision for Bentley. And now we are going further, towards a more sustainable future. In 2020 Bentley Motors launched the ‘Beyond100’ Manifesto, which outlines our intention to become the most sustainable luxury Automotive brand in the world.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
google_api_key=GOOGLE_API_KEY
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.35, max_tokens=600,google_api_key=google_api_key)

In [23]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [24]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [25]:
response = rag_chain.invoke({"input": "What is Bentayga?"})
print(response["answer"])

The Bentayga is Bentley's first SUV model and was launched in 2015. It was designed to offer high levels of luxury and performance, typical of the Bentley brand.  The Bentayga was named after Roque Bentayga, a mountain located in Gran Canaria, Spain. 

